In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
import numpy as np
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [2]:
# data prep from previous moduel
file=r'C:/Users/DELL/Desktop/Edvancer Old Study Material/Data/census_income.csv'

ci_train=pd.read_csv(file)

In [3]:
ci_train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,Y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
pd.crosstab(ci_train['education'],ci_train['education.num'])

education.num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
education,,,,,,,,,,,,,,,,
10th,0,0,0,0,0,933,0,0,0,0,0,0,0,0,0,0
11th,0,0,0,0,0,0,1175,0,0,0,0,0,0,0,0,0
12th,0,0,0,0,0,0,0,433,0,0,0,0,0,0,0,0
1st-4th,0,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5th-6th,0,0,333,0,0,0,0,0,0,0,0,0,0,0,0,0
7th-8th,0,0,0,646,0,0,0,0,0,0,0,0,0,0,0,0
9th,0,0,0,0,514,0,0,0,0,0,0,0,0,0,0,0
Assoc-acdm,0,0,0,0,0,0,0,0,0,0,0,1067,0,0,0,0
Assoc-voc,0,0,0,0,0,0,0,0,0,0,1382,0,0,0,0,0


In [5]:
ci_train.drop(['education'],axis=1,inplace=True)

In [6]:
ci_train['Y'].value_counts().index

Index([' <=50K', ' >50K'], dtype='object')

In [7]:
ci_train['Y']=(ci_train['Y']==' >50K').astype(int)

In [8]:
cat_cols=ci_train.select_dtypes(['object']).columns

In [9]:
cat_cols

Index(['workclass', 'marital.status', 'occupation', 'relationship', 'race',
       'sex', 'native.country'],
      dtype='object')

In [10]:
ci_train.shape

(32561, 14)

In [11]:
for col in cat_cols:
    freqs=ci_train[col].value_counts()
    k=freqs.index[freqs>500][:-1]
    for cat in k:
        name=col+'_'+cat
        ci_train[name]=(ci_train[col]==cat).astype(int)
    del ci_train[col]
    print(col)

workclass
marital.status
occupation
relationship
race
sex
native.country


In [12]:
ci_train.shape

(32561, 39)

In [13]:
ci_train.isnull().sum()

age                                   0
fnlwgt                                0
education.num                         0
capital.gain                          0
capital.loss                          0
hours.per.week                        0
Y                                     0
workclass_ Private                    0
workclass_ Self-emp-not-inc           0
workclass_ Local-gov                  0
workclass_ ?                          0
workclass_ State-gov                  0
workclass_ Self-emp-inc               0
marital.status_ Married-civ-spouse    0
marital.status_ Never-married         0
marital.status_ Divorced              0
marital.status_ Separated             0
occupation_ Prof-specialty            0
occupation_ Craft-repair              0
occupation_ Exec-managerial           0
occupation_ Adm-clerical              0
occupation_ Sales                     0
occupation_ Other-service             0
occupation_ Machine-op-inspct         0
occupation_ ?                         0


In [14]:
x_train=ci_train.drop(['Y'],1)
y_train=ci_train['Y']

C:\Users\DELL\AppData\Local\Temp\ipykernel_14848\1215536160.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train=ci_train.drop(['Y'],1)


In [15]:
from sklearn.model_selection import RandomizedSearchCV

In [16]:
params={'class_weight':[None,'balanced'],
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20],
            'min_samples_split':[2,5,10,15,20]
        }

In [17]:
2*2*8*6*5

960

In [18]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
clf=DecisionTreeClassifier()

In [20]:
random_search=RandomizedSearchCV(clf,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=10)

In [21]:
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(),
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'criterion': ['entropy', 'gini'],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              20]},
                   scoring='roc_auc')

In [22]:
random_search.best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=50, min_samples_leaf=20, min_samples_split=10)

In [23]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Model validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [24]:
report(random_search.cv_results_,5)

Model with rank: 1
Model validation score: 0.891 (std: 0.00629)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 20, 'max_depth': 50, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 2
Model validation score: 0.888 (std: 0.00655)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 15, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 3
Model validation score: 0.885 (std: 0.00643)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 15, 'max_depth': 70, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 4
Model validation score: 0.881 (std: 0.00894)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 20, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 5
Model validation score: 0.878 (std: 0.00617)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 10, 'max_depth': 20, 'criterion': 'gini', 'class_weight': None}



In [25]:
dtree=random_search.best_estimator_

In [26]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=50, min_samples_leaf=20, min_samples_split=10)

In [27]:
dotfile = open("mytree.dot",'w')

tree.export_graphviz(dtree,out_file=dotfile,
                    feature_names=x_train.columns,
                    class_names=["0","1"],
                    proportion=True)

dotfile.close()

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
clf = RandomForestClassifier()

In [30]:
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True,False],
              'class_weight':[None,'balanced'],
                  'criterion':['entropy','gini'],
                  'max_depth':[None,5,10,15,20,30,50,70],
                  'min_samples_leaf':[1,2,5,10,15,20],
                  'min_samples_split':[2,5,10,15,20]
    
}

In [31]:
x_train.shape

(32561, 38)

In [32]:
960*6*6*2

69120

In [ ]:
n_iter_search = 10

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                  n_iter=n_iter_search,scoring='roc_auc',cv=5)

random_search.fit(x_train, y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
report(random_search.cv_results_,5)

In [ ]:
rf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=50,
                         max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07,
                          min_samples_leaf=10, min_samples_split=20, min_weight_fraction_leaf=0.0,
                         n_estimators=300, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False)

### Feature Importance

In [ ]:
feat_imp_df=pd.DataFrame({'features':x_train.columns,'importance':rf.feature_importances_})

feat_imp_df.sort_values('importance',ascending=False)

### Partial Dependence Plot

In [ ]:
var_name='education.num'

preds=rf.predict_proba(x_train)[:,1]

In [ ]:
var_data=pd.DataFrame({'var':x_train[var_name],'response':preds})

In [ ]:
import seaborn as sns

sns.lmplot(x='var',y='response',data=var_data,fit_reg=False)

In [ ]:
import statsmodel.api as sm
smooth_data=sm.nonparametric.lowess(var_data['response'],var_data['var'])

In [ ]:
df=pd.DataFrame({'response':smooth_data[:,1],var_name:smooth_data[:,0]})

sns.lmplot(x=var_name,y='response',data=df,fit_reg=False)